In [1]:
pip install llamaindex
pip install transformers
pip install faiss-cpu  # vector database

In [ ]:
articles = [
    {
        "url": "https://blog.langchain.dev/enhancing-rag-based-applications-accuracy-by-constructing-and-leveraging-knowledge-graphs/",
        "content": """Enhancing RAG-Based Applications Accuracy by Constructing and Leveraging Knowledge Graphs...
                      [full content here]"""
    },
    {
        "url": "https://blog.langchain.dev/graph-based-metadata-filtering-for-improving-vector-search-in-rag-applications/",
        "content": """Graph-Based Metadata Filtering for Improving Vector Search in RAG Applications...
                      [full content here]"""
    }
]

documents = [article['content'] for article in articles]


In [ ]:
from llamaindex import GPTIndexer, FAISSRetriever
from transformers import AutoTokenizer, AutoModel

# Initialize the tokenizer and model for embeddings
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings

# Prepare document embeddings
document_embeddings = get_embeddings(documents)

# Initialize FAISS retriever
retriever = FAISSRetriever(vector_dim=document_embeddings.shape[1])
retriever.add_embeddings(document_embeddings, documents)


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Initialize the T5 model and tokenizer
model_name = "t5-base"
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)

def generate_answer(context, question):
    input_text = "context: {} question: {}".format(context, question)
    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")
    outputs = t5_model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


In [ ]:
def rag_system(question):
    # Retrieve relevant documents
    retrieved_docs = retriever.retrieve(question, top_k=3)
    context = " ".join(retrieved_docs)

    # Generate an answer based on retrieved context
    answer = generate_answer(context, question)
    return answer

# Example usage
question = "How does constructing and leveraging knowledge graphs enhance RAG-based applications accuracy?"
answer = rag_system(question)
print("Q:", question)
print("A:", answer)